In [21]:
#Essentials
import numpy as np
import pandas as pd
import pickle
import re
import datetime as dt
from termcolor import colored

#SQL related
import psycopg2
import sqlite3
from sqlalchemy import create_engine
import pandas.io.sql as pd_sql


#Visualization
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

#Geography magic
import geocoder
import googlemaps
import geopy.distance as gd
import geopandas as gpd
import shapely
from shapely.geometry import Point
import fiona
import pysal as ps
import gmplot

import folium
from folium import plugins

from IPython.display import HTML

# from mpl_toolkits.basemap import Basemap

%matplotlib inline

In [22]:
# Set up the database connection
conn = psycopg2.connect(dbname="taxi_rides_db", user="auste_m")

In [23]:
taxi_weather_query = """SELECT TR.trip_start_timestamp,
                                TR.trip_end_timestamp,
                                TR.trip_seconds,
                                TR.trip_miles,
                                CASE WHEN TR.tolls IS NULL THEN TR.fare ELSE TR.fare + TR.tolls END as fare_with_tolls,
                                TR.payment_type,
                                TR.pickup_centroid_latitude,
                                TR.pickup_centroid_longitude,
                                TR.pickup_centroid_location,
                                TR.dropoff_centroid_latitude,
                                TR.dropoff_centroid_longitude,
                                TR.dropoff_centroid_location,
                                CW.avg_daily_wind_speed,
                                CW.avg_temp_f,
                                CASE WHEN CW.fog = 1 OR CW.heavy_fog = 1 OR ice_fog = 1 OR mist = 1 THEN 1 ELSE 0 END as fog,
                                CASE WHEN CW.rain = 1 OR CW.drizzle = 1 OR CW.hail = 1 THEN 1 ELSE 0 END as rain,
                                CASE WHEN CW.snow = 1 OR CW.freezing_rain = 1 OR drifting_snow = 1 THEN 1 ELSE 0 END as snow
                                
                        FROM taxi_rides_clean as TR
                        LEFT JOIN chicago_weather as CW
                        ON CAST(TR.trip_start_timestamp AS DATE) = CAST(date AS DATE)
                        WHERE trip_start_timestamp BETWEEN '2016-01-01' AND '2016-12-31';""" #'2017-07-31'

In [24]:
#approx. 30min to load the whole of 2016 data
sql_load = pd.read_sql_query(taxi_weather_query, con=conn)

In [25]:
#Let's drop all the records with a missing lat / long in either pickup or dropoff
taxi_weather_clean = sql_load.dropna(subset=['pickup_centroid_latitude', 'pickup_centroid_longitude', 'dropoff_centroid_latitude'
                                                    , 'dropoff_centroid_longitude', 'trip_seconds', 'trip_miles', 'fare_with_tolls'])

In [26]:
#Let's get rid of the zeroes
taxi_weather_clean = taxi_weather_clean[(taxi_weather_clean['fare_with_tolls'] != 0.0) 
                                          & (taxi_weather_clean['trip_seconds'] != 0.0) 
                                          & (taxi_weather_clean['trip_miles'] != 0.0)]

In [27]:
taxi_weather_clean.count()[0]

13354486

## Plotting fun!

In [28]:
#replace pickup_centroid_location and dropoff_centroid_location columns with point coordinates using shapely
pickup_geopoint = list(zip(taxi_weather_clean['pickup_centroid_longitude'], taxi_weather_clean['pickup_centroid_latitude']))
dropoff_geopoint = list(zip(taxi_weather_clean['dropoff_centroid_longitude'], taxi_weather_clean['dropoff_centroid_latitude']))

taxi_weather_clean['pickup_centroid_location'] = pickup_geopoint
taxi_weather_clean['dropoff_centroid_location'] = dropoff_geopoint

taxi_weather_clean['pickup_centroid_location'] = taxi_weather_clean['pickup_centroid_location'].apply(Point)
taxi_weather_clean['dropoff_centroid_location'] = taxi_weather_clean['dropoff_centroid_location'].apply(Point);


In [29]:
#Make a geopandas dataframe
taxi_weather_gdf = gpd.GeoDataFrame(taxi_weather_clean, geometry='pickup_centroid_location')

## Working map with origin & destination and a line inbetween

In [30]:
# Place map
gmap = gmplot.GoogleMapPlotter.from_geocode('Chicago, USA')

latitudes = [41.88890145, 41.87624705]
longitudes = [-87.6264253897545, -87.6530482505327]

# Plotting the dots
gmap.scatter(latitudes, longitudes, 'magenta', size=200, marker=False)

gmap.plot(latitudes, longitudes,  
           'blue', edge_width = 2.5) 

# Draw
gmap.draw("./Chicago_taxi_route.html")

# taxi_weather_gdf.plot(figsize=(10, 10), alpha=0.5, edgecolor='c')
# plt.title('Pickup locations')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')


IndexError: list index out of range

In [ ]:
%%HTML
<iframe width="80%" height="350" src="./Chicago_taxi_route.html"></iframe>

## Plotting the demand

In [37]:
chicago_map = folium.Map(location = [taxi_weather_clean['pickup_centroid_latitude'].mean(), 
                                     taxi_weather_clean['pickup_centroid_longitude'].mean()], 
                         zoom_start = 13, tiles='Stamen Terrain')

In [38]:
chicago_map

In [ ]:
locations = [list(loc) for loc in 
             taxi_weather_clean[['pickup_centroid_latitude', 'pickup_centroid_longitude']].sample(3000000).as_matrix()]

chicago_map.add_child(plugins.HeatMap(data=locations, name='taxi_demand_map', min_opacity=0.04, radius=10,
                     gradient={0.3: 'lime', 0.65: 'blue', 1: 'magenta'}))

chicago_map.save('Chicago_taxi_demand.html')
chicago_map

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
